In [1]:
import pandas as pd

import torch

from torch_geometric.utils import to_scipy_sparse_matrix

from data import get_sbm
from stats import compute_statistics
from utils import get_results_path


In [ ]:
# Dataset parameters
cluster_std = 0.4
n_features = 16
num_classes = 10
num_nodes_per_class = 100
num_nodes = num_nodes_per_class * num_classes
block_sizes = torch.tensor([num_nodes_per_class] * num_classes)


In [2]:
stats_path = get_results_path() / "statistics_synthetic.csv"
df = pd.DataFrame()


In [4]:
# Get values for random graphs

for edge_num_self, edge_num_other in [
    (6, 3),
    (6, 6),
    (3, 6),
    (1, 9),
]:
    name = f"p_ii={edge_num_self}/100, p_ij={edge_num_other}/100"

    prob_matrix = torch.full(
        (num_classes, num_classes),
        edge_num_other / (num_nodes - num_nodes_per_class),
    )
    edge_prob_self = edge_num_self / block_sizes
    edge_probs = (prob_matrix * (1 - torch.eye(num_classes))) + torch.diag(
        edge_prob_self
    )

    data = get_sbm(
        block_sizes=block_sizes,
        edge_probs=edge_probs,
        centers=torch.eye(num_classes),
        cluster_std=cluster_std,
        n_features=n_features,
        seed=0,
    )

    print(f"Processing {name}...")

    A = to_scipy_sparse_matrix(data.edge_index)

    df = compute_statistics(A, name, df)
    df.to_csv(stats_path)


In [6]:
df[["1_mean", "1_std", "1_expected", "2_mean", "2_std", "2_expected"]]


1_mean     1_std  1_expected    2_mean  \
Dataset                Self-loops                                             
p_ii=6/100, p_ij=3/100 True        0.113076  0.046143    0.101329  0.095121   
                       False       0.000000  0.000000    0.000000  0.102750   
p_ii=6/100, p_ij=6/100 True        0.081962  0.024870    0.075942  0.071819   
                       False       0.000000  0.000000    0.000000  0.076469   
p_ii=3/100, p_ij=6/100 True        0.109682  0.036987    0.099990  0.094211   
                       False       0.000000  0.000000    0.000000  0.102021   
p_ii=1/100, p_ij=9/100 True        0.100141  0.032885    0.091475  0.086232   
                       False       0.000000  0.000000    0.000000  0.092780   

                                      2_std  2_expected  
Dataset                Self-loops                        
p_ii=6/100, p_ij=3/100 True        0.010718    0.092278  
                       False       0.012392    0.101659  
p_ii=6/100, p_ij=6/100 True        0.005457    0.070614  
                       False       0.006172    0.075979  
p_ii=3/100, p_ij=6/100 True        0.009105    0.091537  
                       False       0.010417    0.100760  
p_ii=1/100, p_ij=9/100 True        0.008388    0.083918  
                       False       0.009371    0.091605

In [7]:
df[
    [
        "3_mean",
        "3_std",
        "4_mean",
        "4_std",
        "5_mean",
        "5_std",
        "6_mean",
        "6_std",
    ]
]


3_mean     3_std    4_mean     4_std  \
Dataset                Self-loops                                           
p_ii=6/100, p_ij=3/100 True        0.026572  0.003723  0.018963  0.002708   
                       False       0.002480  0.002471  0.020318  0.003397   
p_ii=6/100, p_ij=6/100 True        0.015522  0.001438  0.010998  0.001345   
                       False       0.001288  0.001205  0.011642  0.001664   
p_ii=3/100, p_ij=6/100 True        0.024829  0.002786  0.017524  0.002066   
                       False       0.000788  0.001318  0.019002  0.002830   
p_ii=1/100, p_ij=9/100 True        0.021038  0.002454  0.014793  0.001780   
                       False       0.000746  0.001177  0.015925  0.002363   

                                     5_mean     5_std    6_mean     6_std  
Dataset                Self-loops                                          
p_ii=6/100, p_ij=3/100 True        0.008560  0.001732  0.006008  0.001524  
                       False       0.002230  0.001295  0.006021  0.001628  
p_ii=6/100, p_ij=6/100 True        0.004181  0.000745  0.002945  0.000675  
                       False       0.001206  0.000530  0.002965  0.000722  
p_ii=3/100, p_ij=6/100 True        0.007190  0.001110  0.004904  0.001051  
                       False       0.000956  0.000594  0.005092  0.001260  
p_ii=1/100, p_ij=9/100 True        0.005784  0.000926  0.003968  0.000881  
                       False       0.000923  0.000519  0.004103  0.001028